# Apply NLMpy's Random Cluster algorithm to study sites

Script by the original authors to generate a neutral landscape model using the modified randon clusters algorithm can be found here [here](https://besjournals.onlinelibrary.wiley.com/action/downloadSupplement?doi=10.1111%2F2041-210X.12308&attachmentId=112210370)

Original [paper](http://doi.org/10.1111/2041-210X.12308)

In [ ]:
from pathlib import Path
import os
import subprocess
import zipfile
import re

import requests

from osgeo import gdal
import rasterio
from rasterio.plot import show

import numpy as np
import pandas as pd

from nlmpy import nlmpy

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
pwd = os.getcwd().split('/')[-1]
in_landcover_nlms = pwd == 'landcover-nlms'
TMP_DIR = Path('../tmp') if in_landcover_nlms else Path('tmp')
OUTPUT_DIR = Path('../outputs') if in_landcover_nlms else Path('outputs')
PLOTS_DIR = OUTPUT_DIR / 'plots'
PLOTS_DIR.mkdir(exist_ok=True)

The script used by Etherington, Holland and O'Sullivan to produce Fig. 2 in their original [paper](http://doi.org/10.1111/2041-210X.12308) is retrieved and shown below:

In [ ]:
script_url = 'https://besjournals.onlinelibrary.wiley.com/action/downloadSup'\
             'plement?doi=10.1111%2F2041-210X.12308&attachmentId=112210370'
print(requests.get(script_url).text)

## Play with NLMpy

Objectives are to:
1. Demonstrate how we can retrieve grids with proportions of landcover matching specified proportions 
2. Quantify error over possible solutions

In [ ]:
nRow = 500
nCol = 500
rc_array = nlmpy.randomClusterNN(nRow, nCol, 0.58)

In [ ]:
plt.hist(rc_array)

In [ ]:
plt.matshow(rc_array)

In [ ]:
rc_classified_even_array = nlmpy.classifyArray(rc_array, [0.5, 0.5])

In [ ]:
plt.hist(rc_classified_even_array)

In [ ]:
plt.matshow(rc_classified_even_array)

In [ ]:
rc_classified_even_array[:10, :10]

In [ ]:
unique, counts = np.unique(rc_classified_even_array, return_counts=True)
n = nRow*nCol
dict(zip(unique, counts/float(n)))

In [ ]:
rc_classified_uneven_array = nlmpy.classifyArray(rc_array, [0.1, 0.8, 0.1])

In [ ]:
plt.hist(rc_classified_uneven_array)

In [ ]:
unique, counts = np.unique(rc_classified_uneven_array, return_counts=True)
n = nRow*nCol
dict(zip(unique, counts/float(n)))

In [ ]:
rc_classified_uneven_array_shuffled = nlmpy.classifyArray(rc_array, [0.1, 0.1, 0.8])

In [ ]:
plt.hist(rc_classified_uneven_array_shuffled)

In [ ]:
unique, counts = np.unique(rc_classified_uneven_array_shuffled, return_counts=True)
n = nRow*nCol
dict(zip(unique, counts/float(n)))

**So** the list passed as the `weights` argument to nlmpy.classifyArray can be a list of proportions, and the class labels in the returned classified array correspond to the index of each provided wight.

E.g. if we give the weights list `[0.1, 0.8, 0.1]` nlmpy.classifyArray will return an array where 10% of the elements are labelled `0`, 80%  are labelled `1` and 10% are labelled `2`

## Load DEM data from a GeoTiff file
Etherington, Holland and O'Sullivan demonstrated `nlmpy`'s capability for integrating different NLMs for different elevation ranges using ASCIIGrid data. I will perform a similar analysis using a GeoTIFF file containing a DEM (STRM 1-second arc data) obtained using the notebook `download_site_elevation_data.html`.

To extract a numpy array from the GeoTIFF file, we will make use of the [`rasterio`](https://pypi.org/project/rasterio) package.

In [ ]:
pwd = os.getcwd().split('/')[-1]
OUTPUT_DIR_ROOT = (Path('../outputs') 
                   if pwd == 'landcover-nlms' else Path('outputs'))
TMP_DIR = Path('../tmp/') if pwd == 'landcover-nlms' else Path('tmp')
test_site = 'navarres'

In [ ]:
navarres_dem_file = OUTPUT_DIR_ROOT / test_site / 'hydrocorrect_dem.tif'
with rasterio.open(navarres_dem_file) as src:
    dem = src.read()

In [ ]:
show(dem)

In addition to helping us to quickly plot our raster data, `rasterio` allows us to work with the DEM as a simple `numpy` array:

In [ ]:
print('type of raster object:', type(dem))
print('shape of raster array:', dem.shape)
print('array data type:', dem.dtype)
print('a slice of the data:')
print(dem[0,100:110,100:110])

Note that the first index of the returned `numpy` array describes the raster _band_. In the case of a DEM there is only one band: elevation in meters. However, as we shall see different types of raster data include several bands. 

## Load a satellite image to guide our design
I'm imagining a plot which has satellite DEM, satelite image and proposed NLM next to each other for illustrative purposes. Satellite image will be needed to get an impression of land cover at that elevation in the present day.

Focussing on the Navarrés study site as a starting point, we note it has the following lat/lon coordinates:
- nav_lat = 39.1
- nav_lon = -0.683333

Steps to obtain landsat imagery
- Go to USGS Earth Explorer 
- Search for the lat/lon coordinates given above, and a timeframe likely to have clear weather (i.e. a clear day)
- Click on 'Sata Sets' tab
- Select dataset Landsat > Landsat Collection 1 Level-1 > Landsat 8 OLI/TIRS C1 Level-1
- Click 'Results'
- Look for a thumbnail with little cloud cover
- Click the 'download' icon for that scene
- Select 'LandsatLook Images with Geographic Reference' option

This downloads a zip containing, along with [other things](https://landsat.usgs.gov/landsatlook-images),  a natural color georeferenced image of the scene. The other things in the zip are a 'Quality' image (suffix \_QB) used for e.g. detecting clouds, and 'Thermal' image (suffix \_TIR) which we won't use in our analyses.


## Clip landsat image to the same extent as the DEM

### Extract LandsatLook image from zip file
The LandsatLook images come in `.zip` files containing other data. We'll first temporarily extract the specific file we need to work with.

In [ ]:
def extract_landsatlook_tif(zipfile_name, output_dir=None, output_name=None):
    """Extract a georeferenced LandsatLook tif from Earth Explorer zip file.
    
    Given a 'LandsatLook Images with Geographic Reference' zip file downloaded 
    from USGS Earth Explorer https://earthexplorer.usgs.gov, select the 
    Natural Colour .tif file and extract it to the specified output_dir, 
    naming it if given. Essentially this amounts to finding the file in the 
    zip archive which doesnt have a suffix indicating it's a 'Quality' image
    (suffix _QB) or a 'Thermal' image (suffix _TIR).
    
    Args:
        zipfile_name (str): Zipfile to be processed.
        output_dir (Optional[str]): Path to directory in which to save 
            resulting tif file.
        output_name (Optional[str]): Name to give output file.        
    
    Returns:
        str: Path to the output file.
    """
    
    with zipfile.ZipFile(zipfile_name, 'r') as z:
        all_files = z.namelist()
        natural_colour_files = [f for f in all_files 
                                if not(re.match(r'.*_QB.tif|.*_TIR.tif', f))]
        if len(natural_colour_files) < 1:
            raise ValueError('No Natural Colour .tif found in .zip')
        elif len(natural_colour_files) > 1:
            raise ValueError('{0} Natural Colour .tif files found in .zip '\
                             'when only one expected. Check file.')
        else:
            # Only if exactly one natural colour file identified, extract it
            nc_file = natural_colour_files[0]
            z.extract(nc_file, path=output_dir)
            if (output_name and output_dir):
                created_file = os.path.join(output_dir, output_name)
                os.rename(os.path.join(output_dir, nc_file), created_file)
            elif output_dir:
                created_file = os.path.join(output_dir, nc_file)
            elif output_name:
                created_file = output_name
                os.rename(nc_file, created_file)
            else:
                created_file = nc_file
            
            return created_file  

### Use `gdalwarp` to do the clipping

In [ ]:
def match_tif_extents(tif_to_match, tif_to_cut, output_name):
    """Use one tif file to crop another to the same extent and projection.
    
    Args:
        tif_to_match (str): Name of a tif file to use as an example whose
            extent and crs we want the other file to match.
        tif_to_cut (str): Name of a tif file to crop and reproject to match
            tif_to_match.
        output_name (str): Name of resulting cropped and reprojected 
            tif file.
            
    Returns:
        str: Path to the output file.   
    """
    src_data = gdal.Open(tif_to_match, gdal.GA_ReadOnly)
    wkt = src_data.GetProjection()
    geoTransform = src_data.GetGeoTransform()
    minx = geoTransform[0]
    maxy = geoTransform[3]
    maxx = minx + geoTransform[1] * src_data.RasterXSize
    miny = maxy + geoTransform[5] * src_data.RasterYSize
    src_data = None
    
    with open('tmp.prj', 'w') as prj:
        # write a temporary well known text file to be used by gdalwarp
        prj.write(wkt)
       
    # specify parameters to be passed to gdalwarp in an external process:
    param = ['gdalwarp', tif_to_cut, output_name, '-overwrite',
             '-t_srs', 'tmp.prj',
             '-te', str(minx), str(miny), str(maxx), str(maxy)]
    
    cmd = ' '.join(param)
    process = subprocess.check_call(cmd, shell=True)
    
    os.remove('tmp.prj')
    
    return output_name

In [ ]:
nav_nc = extract_landsatlook_tif(
    'data/LC08_L1TP_199033_20170730_20170811_01_T1.zip',
    'data/',
    'navarres_tmp.tif'
)
print(nav_nc)

In [ ]:
match_tif_extents(str(navarres_dem_file), nav_nc, 'data/navarres_lsat.tif')

In [ ]:
with rasterio.open('data/navarres_lsat.tif') as src:
    lsat = src.read()
show(lsat)

This time we find that the returned numpy array has three bands corresponding to Red, Green and Blue which the `rasterio.plot.show` function has conveniently worked out how to show in one image.

In [ ]:
print('Shape of landsat image\'s numpy array:', lsat.shape)

print('\nRed band slice:')
print(lsat[0,100:110,100:110])

print('\nBlue band slice:')
print(lsat[1,100:110,100:110])

print('\nGreen band slice:')
print(lsat[2,100:110,100:110])

## Plot DEM and satellite image side-by side
This will be a useful plot when deciding whether land cover proposed by Neutral Landscape model appears appropriate

In [ ]:
fs=18
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize=(12,9) )
show(dem, ax=ax1)
ax1.set_axis_off()
ax1.set_title('SRTM 1-sec arc', fontsize=fs)

show(lsat, ax=ax2)
ax2.set_axis_off()
ax2.set_title('Landsat 8', fontsize=fs)

plt.tight_layout()

## Identify initial condition land cover proportions for each study site

### Determine initial vegetation proportions from pollen analysis
Enforce the rule that the proportional area of land occupied by each land cover class in the NLM corresponds to the proportion of pollen corresponding to that vegetation type found in the sediment core for that study site.

Note that this will need to be refined by incorporating the LRA, i.e. need to translate pollen _count_ to landscape _proportion_ in a more principled way. However, this will come later.

#### Load pollen time series

In [ ]:
site_meta = (
    pd.read_csv(OUTPUT_DIR_ROOT / 'site_metadata.csv').set_index('sitecode')
)

Load land cover proportions from time series

In [ ]:
ts = (
    pd.concat([
        pd.read_csv(OUTPUT_DIR_ROOT / site / 'lct_pct_ts.csv')
        .assign(sitecode=site) 
        for site in site_meta.index
    ])
    .set_index(['sitecode', 'agebp'])
    .sort_index()
    .filter(regex='^pct_*')  # Drop columns corresponding to derivatives
    .rename(mapper=lambda s: s.replace('pct_', ''), axis='columns')
    .divide(100)  # Convert percentage to proportion
)

For now we'll use only timeseries corresponding to the _proportion_ of counted pollen belonging to each landcover type.

In [ ]:
assert (ts.sum(1) - 1 < 0.00001).all(), (
    'Land cover proportions sould sum to 1.'
)

ts

Our study sites are as follows:

In [ ]:
print('# datapoints for included study sites:', ts.shape[0])
print('# features for included study sites:', ts.shape[1])

#### Identify $t_0$ years for each study site

In [ ]:
ssite_t0 = pd.Series({'navarres': 7000, 
                      'charco_da_candieira': 6500, 
                      'atxuri': 5000, 
                      'monte_areo_mire': 7300, 
                      'algendar': 5000, 
                      'san_rafael': 5000}).rename('t0')

In [ ]:
ssite_t0

These `t0` values correspond to the date at which it is belied humans first started practicing agriculture at each study site. 

#### Find land cover proportions at $t_0$

In [ ]:
t0_lct_props = ts.loc[zip(ssite_t0.index, ssite_t0)]

In [ ]:
for (site_code, year), lct_props in t0_lct_props.iterrows():
    print(site_code)

In [ ]:
fig, ax = plt.subplots(figsize=(9,5))
t0_lct_props.plot(kind='bar', stacked=True, ax=ax)
plt.legend(bbox_to_anchor=(.28,1.03), ncol=2);

Save initial conditions to a temporary file

In [ ]:
t0_lct_props.to_csv(TMP_DIR / 't0_lct_props.csv')

### Specify NLM model

- Need to account for elevation, incorporating DEM. Account for existance of treeline
- Also need to enforce the potentially conflicting criteria that the overall proportion of landcover should match the proportions specified in `t0_lct_props`.
- The methods I'll be using are `nlmpy.randomClusterNN` to generate some random clusters with the right size, for above and below the treeline, `nlmpy.classifyArray` to assign clusters to the land cover classes with a weighting proportional to each lct's land cover proportions, and `np.where` to decide which part of the map will be characterised by the lowland NLM, and which part the highland.

In [ ]:
try:
    t0_lct_props
except NameError:
    t0_lct_props = (
        pd.read_csv(TMP_DIR / 't0_lct_props.csv')
        .set_index(['sitecode', 'agebp'])
    )

A quick review of `nlmpy` capabilities

In [ ]:
#@interact(perc_thresh=0.5, prop1=0.5, prop2=0.5)
def make_500by500_MRC_array(perc_thresh, prop1, prop2):
    nlm = nlmpy.randomClusterNN(500, 500, perc_thresh)
    return plt.matshow(nlmpy.classifyArray(nlm, [prop1, prop2]), cmap='Set3')
    
make_500by500_MRC_array(0.6, 0.2, 0.8)

#### Set up a test to evaluate the number of iterations required to gain acceptable solution

The algorithm for obtaining a random clusters map involves first generating clusters using `nlmpy.randomClusterNN` before llocating these to categories matching desired proportions using `nlmpy.classifyArray`. This means there is some variance in how successful the algorithm is at matching the reuested proportions. Below I run a test to evaluate performance.

In [ ]:
def generate_test_array(nRow, nCol, target_proportions, perc_threshold):
    test = nlmpy.randomClusterNN(nRow, nCol, perc_threshold)
    test = nlmpy.classifyArray(test, target_proportions)
    return test   

In [ ]:
def test_random_clusters_accuracy(target_proportions, perc_threshold, no_iterations=100):
    """Return overall root mean square error.
    
    Construct `no_iterations` landscapes using the target proportions and 
    percolation threshold given. Compare the landcover proportions in the 
    resulting landscape with the target proportions, constructing a root mean
    square error as a score.
    
    """
    N = 500 # approximate size of DEMs for study site
    N2 = N*N
    res = np.empty((no_iterations, len(target_proportions)))
    for i in range(no_iterations):
        # make a test array
        a = generate_test_array(N, N, target_proportions, perc_threshold)
        output_props = []
        for val in np.unique(a):
            # calculate the proportion of each land cover type in test array
            output_props.append((a==val).sum()/float(N2))
        # find square of differences between test array's proportions and
        # target proportions
        try:
            res[i,:] = np.power(np.array(target_proportions) - np.array(output_props), 2)        
        except ValueError:
            'target proportions: {0}\nperc_threshold: {1}\nno iterations: {2}\n'\
            'output proportions: {3}\n'.format(target_proportions, 
                                               perc_threshold, 
                                               no_iterations, 
                                               output_props)

        
    # sum up errors across all proportions
    res = res.sum()/float(N2)
    # take square root
    res = np.sqrt(res)        

    return res

In [ ]:
def get_MRC_RMSE(result_array, target_props):
    """Find error in MRC array vs target proportions.
    
    Calculates the proportion of an array allocated to each class and 
    calculates the RMS error across classes to give the resulting array 
    a score on the basis of its ability to reproduce the required land
    cover characteristics.
    """
    target_props = np.array(target_props)
    # total number of entries
    N = result_array.size
    classes = np.unique(result_array)
    if classes.shape != target_props.shape:
        raise ValueError('# classes in array don\'t match # requested '\
                         'proportions.')
    result_props = []
    for val in classes:
        # calculate proportion of array belonging to each class
        result_props.append((result_array==val).sum()/float(N))
    result_props = np.array(result_props)
    #rmse calculation
    rmse = np.sqrt(np.power(target_props-result_props,2).sum()/target_props.shape[0])   
    return rmse

Specify proportion scenarios

In [ ]:
prop_scenarios = {
    '2_similar': [0.5, 0.5],
    '2_different': [0.1, 0.9],
    '4_similar': [0.25, 0.25, 0.25, 0.25],
    '4_different': [0.1, 0.3, 0.5, 0.1],    
}

Specify the different numbers of iterations to try

In [ ]:
iterations = np.arange(10,51)
iterations = iterations[iterations%10==0]
print(iterations)

In [ ]:
percolation_thresholds = np.array([0.1, 0.5, 0.9])

In [ ]:
def test_MRC_algorithm_prop_accuracy():
    df = pd.DataFrame(columns=['prop_scenario', 'no_iterations', 'perc_thresh', 'RMSE'])
    this_idx = 0
    for scenario, proportions in prop_scenarios.iteritems():
        for iteration in iterations:
            for perc in percolation_thresholds:
                r = test_random_clusters_accuracy(proportions, perc, no_iterations=iteration)
                df.loc[this_idx] = [scenario, iteration, perc, r]
                print(df.loc[this_idx])
                this_idx += 1
                
run=False
if run:
    test_MRC_algorithm_prop_accuracy()

It seems the algorithm produces good results in all scenarios tested, except a small number which did really badly. Both of these belong to the `4_different` proportion scenario with a high percolation threshold.

In [ ]:
if run:
    print(df[df.RMSE>1])
    print(df[df.prop_scenario=='4_different'])

Interestingly runs 17, 20 and 23 show the same parameters being run, with fewer iterations, and with good results. I think the lesson here is to keep an eye on the performance of the MRC algorithm in `nlmpy` but not to worry about super high iterations too much.

As can be seen below, the problem seems to be that with very high percolation thresholds, the algorithm finds it difficult to assign any patches to some land cover types, resulting in bad results. It's high percolation thresholds which is the problem, rather than the number of land cover types.

In [ ]:
t = generate_test_array(500, 500, [0.1, 0.9], 0.6)

In [ ]:
np.unique(t)

In [ ]:
plt.matshow(t, cmap='Set3')

#### Transform overall landcover proportions to highland and lowland allocations

Our data gives us the overall proportion of landcover around each study site at $t_0$. However, it would be useful to be able to specify different distributions of landcover above and below the treeline. 

# Define $N^{\text{tot}}$ as the total number of cells in the model. We can separate these cells into components contributed by each of the land-cover classes represented in the model such that

$$
N^{\text{tot}} = N^{\text{tot}}(\mathbf{C}) = \sum_c N_c(\mathbf{C})
$$ 

where we define $N_c(\mathbf{C})$ as the number of cells in class $c \in \{\text{oak forest}, \text{shrubland}\dots\}$. The land-cover class matrix $\mathbf{C}$ has elements $c_{ij}$ encoding the land cover class of the cell in position $(i,j) : i \in (1, \dots, N_y),\, j\in (1, \dots, N_x)$. In symbols,

$$
N_c(\mathbf{C}) = \sum_{i=1}^{N_y} \sum_{j=1}^{N_x} \delta_{c_{ij}, c} 
$$

The total number of cells in class $c$, $N_c(\mathbf{C})$, can be further divided into $N^{\text{hi}}_c(\mathbf{C}, \mathbf{E}; \epsilon) $ and $N^{\text{lo}}_c(\mathbf{C}, \mathbf{E}; \epsilon)$ -- the number of cells in class $c$ which are above and below the treeline respectively. The matrix $\mathbf{E}$ has elements $e_{ij}$ which encode the elevation of the DEM cell in position $(i,j) : i \in (1, \dots, N_y),\, j\in (1, \dots, N_x)$. The parameter $\epsilon$ is the elevation of the treeline and is study site dependent(?). 

$$
N_c(\mathbf{C}) = N^{\text{hi}}_c(\mathbf{C}, \mathbf{E}; \epsilon) + N^{\text{lo}}_c(\mathbf{C}, \mathbf{E}; \epsilon)\\
N^{\text{hi}}_c(\mathbf{C}, \mathbf{E}; \epsilon) = \sum_{i=1}^{N_y} \sum_{j=1}^{N_x} \delta_{c_{ij}, c}\, \Theta(e_{ij} - \epsilon)\\
N^{\text{lo}}_c(\mathbf{C}, \mathbf{E}; \epsilon) = \sum_{i=1}^{N_y} \sum_{j=1}^{N_x} \delta_{c_{ij}, c}\, \left[1 - \Theta(e_{ij} - \epsilon) \right]
$$

In the above, $\Theta(n)$ is the Heaviside step function defined such that 
$$\Theta(n)=
\begin{cases}
    0,\; n<0 \\
    1,\; n\geq0
\end{cases}\,.$$

Since our data is expressed in terms of _proportions_ of land-cover occupied by each class, we define $\rho_c=N_c/N^{\text{tot}}$ (total proportion of land-cover occupied by class $c$), and $\rho^{\text{hi}}_c=N^{\text{hi}}_c/N^{\text{hi}}$ and $\rho^{\text{lo}}_c=N^{\text{lo}}_c/N^{\text{lo}}$ (proportions of cells in class $c$ above and below the treeline respectively). Here $N^{\text{hi}} = \sum_c N^{\text{hi}}_c$ and $N^{\text{lo}} = \sum_c N^{\text{lo}}_c$. Note $\sum_c\rho^{\text{hi}}_c = \sum_c\rho^{\text{lo}}_c =1$. We have

$$
\rho^{\text{hi}}_c(\mathbf{C}, \mathbf{E}; \epsilon) = \frac{1}{N^{\text{hi}}}\sum_{i=1}^{N_y} \sum_{j=1}^{N_x} \delta_{c_{ij}, c}\, \Theta(e_{ij} - \epsilon)\\
\rho^{\text{lo}}_c(\mathbf{C}, \mathbf{E}; \epsilon) = \frac{1}{N^{\text{lo}}} \sum_{i=1}^{N_y} \sum_{j=1}^{N_x} \delta_{c_{ij}, c}\, \left[1 - \Theta(e_{ij} - \epsilon) \right]
$$

We know the value of $\rho_c$ for each $c$ from our data. It will be useful to be able to specify, aspart of our model, the relatively simple proportion of each land-cover type occupying the area above the treeline (e.g. 100% shrubland), and calculate the lowland proportions which preserve our target global land cover proportions, $\rho_c$. We can derive an equationto do this based on the quantities defined above:

$$
N_c = N_c^{\text{hi}} + N_c^{\text{lo}}\\
N^{\text{tot}}\rho_c = N^{\text{hi}}\rho^{\text{hi}}_c + N^{\text{lo}}\rho^{\text{lo}}_c \\
\implies \rho^{\text{lo}}_c(\mathbf{E}, \rho_c, \rho^{\text{hi}}_c; \epsilon) = \frac{N^{\text{tot}}(\mathbf{E}) \,\rho_c- N^{\text{hi}}(\mathbf{E};\epsilon)\,  \rho^{\text{hi}}_c}{N^{\text{lo}}(\mathbf{E};\epsilon)}
$$

Also note 
$$
N^{\text{hi}}(\mathbf{E}; \epsilon) = \sum_{i=1}^{N_y} \sum_{j=1}^{N_x} \Theta(e_{ij} - \epsilon)\\
N^{\text{lo}}(\mathbf{E}; \epsilon) = \sum_{i=1}^{N_y} \sum_{j=1}^{N_x} \left[1 - \Theta(e_{ij} - \epsilon) \right]
$$

Define python functions to perform this calculation

In [ ]:
def get_N_tot(dem_array):
    """Get the number of cells in a DEM passed as a numpy.ndarray"""
    return dem_array.size

def get_N_hi(dem_array, epsilon):
    """Get number of cells above the tree line in dem_array."""
    return dem_array[dem_array>=epsilon].size

def get_N_lo(dem_array, epsilon):
    """Get number of cells below or on the tree line in dem_array."""
    return dem_array[dem_array<epsilon].size

def get_rho_c_lo(dem_array, rho_c, rho_c_hi, epsilon):
    """Calculate rho_c_lo using DEM, data and upland land-cover proportion."""
    num = get_N_tot(dem_array)*rho_c - get_N_hi(dem_array, epsilon)*rho_c_hi
    den = get_N_lo(dem_array, epsilon)
    return float(num)/den

Write a function to get the lowland land-cover proportions given an iterable of highland land-cover proportions, and an iterable of overall land-cover proportions

In [ ]:
def lowland_props(dem_array, rho_c_s, rho_c_hi_s, epsilon):
    """Calculate lowland land-cover proportions given highland and totals.
    
    Args:
        dem_array (numpy.array): Digital Elevation Model used to distinguish
            highland areas from lowland.
        rho_c_s (list of float): Overall land cover proportions, one float per
            land cover type label.
        rho_c_hi_s (list of float): Land cover proportions in the highlands
        epsilon (int): Elevation of the tree line
        
    Returns:
        list of floats: the land cover proportions in the lowlands consistent
            with having rho_c_s overall and rho_c_hi_s in the highlands.        
            
    """
    def get_N_tot(dem_array):
        """Get the number of cells in a DEM passed as a numpy.ndarray"""
        return dem_array.size
    
    def get_N_hi(dem_array, epsilon):
        """Get number of cells above the tree line in dem_array."""
        return dem_array[dem_array>=epsilon].size
    
    def get_N_lo(dem_array, epsilon):
        """Get number of cells below or on the tree line in dem_array."""
        return dem_array[dem_array<epsilon].size
    
    def get_rho_c_lo(dem_array, rho_c, rho_c_hi, epsilon):
        """Calculate rho_c_lo using DEM, data and upland land-cover proportion."""
        num = get_N_tot(dem_array)*rho_c - get_N_hi(dem_array, epsilon)*rho_c_hi
        
        if num < 0:
            raise ValueError('rho_c_lo results in negative lowland proportion' \
                            ' of landcover. Reduce proportion in highland.')
        den = get_N_lo(dem_array, epsilon)
        return float(num)/den    
    
    num_rho_c = len(rho_c_s)
    if num_rho_c != len(rho_c_hi_s):
        raise ValueError('Numbers of provided total land-cover proportions '\
                'and specified highland proportions must match.')
        
    rho_c_lo_s = []
    for i in range(num_rho_c):
        rho_c_lo_s.append(get_rho_c_lo(dem_array, rho_c_s[i], rho_c_hi_s[i], epsilon))   
        
    return rho_c_lo_s

Test on DEM data for Navarres

In [ ]:
navarres_dem_file

In [ ]:
with rasterio.open(navarres_dem_file) as src:
    dem = src.read()

In [ ]:
print('initial DEM shape:', dem.shape)
print('extract the only layer in the dataset')
dem = dem[0, :, :]
print('final DEM shape:', dem.shape)

A look at the distribution of elevations suggests a value of 400 might be a sensible first guess for what we consider uplands. Let's set $\epsilon=400$

In [ ]:
plt.hist(dem.flatten());

In [ ]:
eps = 400

Extracting our land-cover initial conditions from `ssite_t0` we confirm they are in the order deciduous_forest, oak_forest, pine_forest, shrubland in the array returned from the series

In [ ]:
print(t0_lct_props.loc['navarres'], '\n')
rho_c = t0_lct_props.loc['navarres'].values[0, :]
print(rho_c)

First guess at upland land-cover proportions is that all uplands are occupued by shrubland

In [ ]:
rho_c_hi = [0.0, 0.0, 0.0, 1.0]

In [ ]:
rho_c_lo = lowland_props(dem, rho_c, rho_c_hi, eps)
print(rho_c_lo)

In [ ]:
nlm_lo = nlmpy.randomClusterNN(dem.shape[0], dem.shape[1], 0.59)
nlm_lo = (nlmpy.classifyArray(nlm_lo, rho_c_lo)
          .astype('int16'))
# relabel values, assuming same order as in proportion list
nlm_lo = np.where(nlm_lo==0, 1, nlm_lo) # to do with data type, switch to int? 

nlm_hi = nlmpy.randomClusterNN(dem.shape[0], dem.shape[1], 0.5)
nlm_hi = (nlmpy.classifyArray(nlm_hi, rho_c_hi)
          .astype('int16'))
# relabel so shrubland values match up between high and low arrays
nlm_hi = np.where(nlm_hi==0, 3, nlm_hi)

nlm = np.where(dem < eps, nlm_lo, nlm_hi)

In [ ]:
print(np.unique(nlm_lo))
print(np.unique(nlm_hi))

components with proportion 0 aren't represented in classification. Want the following:
- 0: deciduous_forest
- 1: oak_forest
- 2: pine_forest
- 3: shrubland 

In [ ]:
# get landsat image
with rasterio.open('data/navarres_lsat.tif') as src:
    lsat = src.read()

Plot DEM, lsat and nlm together

In [ ]:
fs = 18
fig, (ax1, ax2, ax3) = plt.subplots(nrows=1, ncols=3, figsize=(12, 5))
show(dem, ax=ax1)
ax1.set_axis_off()
ax1.set_title('SRTM 1-sec arc', fontsize=fs)

show(lsat, ax=ax2)
ax2.set_axis_off()
ax2.set_title('Landsat 8', fontsize=fs)

ax3.matshow(nlm, cmap='Set3')
ax3.set_axis_off()
ax3.set_title('NLM', fontsize=fs)

plt.tight_layout()
plt.savefig(PLOTS_DIR / 'navarres_dem_landsat8_nlm.pdf')

## Choose treeline for sites

Different sites are going to need different treelines are some are close to the coast etc. Plot distribution of elevations for each study site:

In [ ]:
def get_raster_array(raster_file: Path) -> np.array:    
    with rasterio.open(raster_file) as src:
        dem = src.read()
    return dem

In [ ]:
bin_edges = np.arange(0, 2200, 200)

In [ ]:
dem_hists = (
    pd.DataFrame({
        site_name: np.histogram(
            get_raster_array(
                OUTPUT_DIR_ROOT / site_name / 'hydrocorrect_dem.tif'
            ),
            bins=bin_edges,
            density=True,
        )[0] 
        for site_name in site_meta.index
    }, index=pd.IntervalIndex.from_breaks(bin_edges))
)

In [ ]:
fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(9, 5))
for i, ax in enumerate(axes.flatten()):
    s = dem_hists.iloc[:, i]
    s.plot.bar(ax=ax)
    ax.set_title(s.name)
    ax.set_xticklabels([x for x in s.index], rotation=45, ha='right')
plt.tight_layout()